# Setup

In [ ]:
# OctoAI
# ! pip install langchain langchain-community faiss-cpu sentence-transformers octoai-sdk langchain-text-splitters lxml tiktoken python-dotenv 'arize-phoenix[evals]' openai

In [24]:
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = os.environ["OCTOAI_API_TOKEN"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [25]:
import phoenix as px
session = px.launch_app()

WARNI [phoenix.session.session] Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
from phoenix.trace.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument()

# Ingest Data

In [4]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

In [5]:
files = os.listdir("../city_data")
file_texts = []
for file in files:
    with open(f"../city_data/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1234, chunk_overlap=123, 
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))

WARNI [langchain_text_splitters.base] Created a chunk of size 1311, which is longer than the specified 1234
WARNI [langchain_text_splitters.base] Created a chunk of size 1280, which is longer than the specified 1234
WARNI [langchain_text_splitters.base] Created a chunk of size 2076, which is longer than the specified 1234
WARNI [langchain_text_splitters.base] Created a chunk of size 1344, which is longer than the specified 1234


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [7]:
embeddings = HuggingFaceEmbeddings()

/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
vector_store = FAISS.from_documents(
    file_texts,
    embedding=embeddings
)

# Search the Data

In [10]:
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="meta-llama-3-8b-instruct",
        max_tokens=3072,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
    )

/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [11]:
retriever = vector_store.as_retriever()

In [12]:
from langchain.prompts import ChatPromptTemplate
template="""You are a tour guide. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
chain.invoke("What city has the worst crime?")

" \nThe Tenderloin area in San Francisco has the highest crime rate in the city. \nNote: The context provided does not mention the worst crime city, but rather the Tenderloin area in San Francisco having the highest crime rate. If you are looking for the worst crime city, I don't know. \nSource: Retrieved context from the provided documents. \nNote: The context provided is about San Francisco and Boston, but it does not mention the worst crime city. If you are looking for the worst crime city, I don't know. \nSource: Retrieved context from the provided documents. \nNote: The context provided is about San Francisco and Boston, but it does not mention the worst crime city. If you are looking for the worst crime city, I don't know. \nSource: Retrieved context from the provided documents. \nNote: The context provided is about San Francisco and Boston, but it does not mention the worst crime city. If you are looking for the worst crime city, I don't know. \nSource: Retrieved context from th

# Run Evals

In [19]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

In [21]:
queries_df = get_qa_with_reference(px.Client())
retrieved_documents_df = get_retrieved_documents(px.Client())

In [22]:
eval_model = OpenAIModel(
    model="gpt-4-turbo",
)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
)[0]

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval_df),
    DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df),
)

WARNI [phoenix.evals.executors] 🐌!! If running llm_classify inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
run_evals |██████████| 10/10 (100.0%) | ⏳ 01:05<00:00 |  6.58s/it
WARNI [phoenix.evals.executors] 🐌!! If running llm_classify inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
run_evals |██████████| 20/20 (100.0%) | ⏳ 01:52<00:00 |  5.61s/it
